In [1]:
import pandas as pd
import numpy as np
import os
import json
from collections import defaultdict

from data_prep_scripts.data_countries import predCountryData 
from data_prep_scripts.data_manipulation import divideRepetitiveBlocks, processBlock 
from data_prep_scripts.field_derivations import deriveRelativesCateg

In [2]:
country_features = predCountryData()

In [3]:
medData = pd.read_excel('Merged File 2.5.19 De-identified.xlsx')
medData['Country of Origin for Father'].replace(76,'Ireland',inplace=True)

In [4]:
familyData1 = medData.iloc[:,47:89]

In [5]:
result = divideRepetitiveBlocks(
    familyData1,
    7,
    categ_cols = [0,2,4,5],
    float_cols = [3],
    derive_fields=[(deriveRelativesCateg,'relative_categ','categ')],
    null_fields=['Relative:','Age at diagnosis'],
    col_trim_begin=3,
    col_trim_end=0,
    lower_case=True
)